# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5c10b1ab20>
├── 'a' --> tensor([[-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194]])
└── 'x' --> <FastTreeValue 0x7f5c10b1aa90>
    └── 'c' --> tensor([[ 1.4982, -1.8341,  0.5934,  1.2519],
                        [-0.8765,  1.0085, -0.1124,  0.6715],
                        [-1.4403, -0.1020, -0.6013, -1.1306]])

In [4]:
t.a

tensor([[-0.2734, -2.8597, -1.3077],
        [-0.0115, -0.3143,  0.4194]])

In [5]:
%timeit t.a

79.2 ns ± 0.593 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5c10b1ab20>
├── 'a' --> tensor([[ 1.6245, -0.2157, -0.6601],
│                   [ 1.4582, -0.7691,  1.2887]])
└── 'x' --> <FastTreeValue 0x7f5c10b1aa90>
    └── 'c' --> tensor([[ 1.4982, -1.8341,  0.5934,  1.2519],
                        [-0.8765,  1.0085, -0.1124,  0.6715],
                        [-1.4403, -0.1020, -0.6013, -1.1306]])

In [7]:
%timeit t.a = new_value

82.4 ns ± 0.125 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2734, -2.8597, -1.3077],
               [-0.0115, -0.3143,  0.4194]]),
    x: Batch(
           c: tensor([[ 1.4982, -1.8341,  0.5934,  1.2519],
                      [-0.8765,  1.0085, -0.1124,  0.6715],
                      [-1.4403, -0.1020, -0.6013, -1.1306]]),
       ),
)

In [10]:
b.a

tensor([[-0.2734, -2.8597, -1.3077],
        [-0.0115, -0.3143,  0.4194]])

In [11]:
%timeit b.a

74.5 ns ± 0.148 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1603, -0.4303,  0.3953],
               [ 0.6644,  0.9413, -0.8884]]),
    x: Batch(
           c: tensor([[ 1.4982, -1.8341,  0.5934,  1.2519],
                      [-0.8765,  1.0085, -0.1124,  0.6715],
                      [-1.4403, -0.1020, -0.6013, -1.1306]]),
       ),
)

In [13]:
%timeit b.a = new_value

604 ns ± 1.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

980 ns ± 1.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.3 µs ± 58.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

170 µs ± 2.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5c10b1a490>
├── 'a' --> tensor([[[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]],
│           
│                   [[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]],
│           
│                   [[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]],
│           
│                   [[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]],
│           
│                   [[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]],
│           
│                   [[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]],
│           
│                   [[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]],
│           
│                   [[-0.2734, -2.8597, -1.3077],
│                    [-0.0115, -0.3143,  0.4194]]])
└── 'x' --> <FastTreeValue 0x7f5be81052b0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37 µs ± 81.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5b647ffdc0>
├── 'a' --> tensor([[-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194],
│                   [-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194],
│                   [-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194],
│                   [-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194],
│                   [-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194],
│                   [-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194],
│                   [-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194],
│                   [-0.2734, -2.8597, -1.3077],
│                   [-0.0115, -0.3143,  0.4194]])
└── 'x' --> <FastTreeValue 0x7f5b647ffe50>
    └── 'c' --> tensor([[ 1.4982, -1.8341,  0.5934,  1.2519],
                        [-0.8765,  1.0085, -0.1124,  0.6715],
                 

In [23]:
%timeit t_cat(trees)

35.3 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.7 µs ± 271 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.4982, -1.8341,  0.5934,  1.2519],
                       [-0.8765,  1.0085, -0.1124,  0.6715],
                       [-1.4403, -0.1020, -0.6013, -1.1306]],
              
                      [[ 1.4982, -1.8341,  0.5934,  1.2519],
                       [-0.8765,  1.0085, -0.1124,  0.6715],
                       [-1.4403, -0.1020, -0.6013, -1.1306]],
              
                      [[ 1.4982, -1.8341,  0.5934,  1.2519],
                       [-0.8765,  1.0085, -0.1124,  0.6715],
                       [-1.4403, -0.1020, -0.6013, -1.1306]],
              
                      [[ 1.4982, -1.8341,  0.5934,  1.2519],
                       [-0.8765,  1.0085, -0.1124,  0.6715],
                       [-1.4403, -0.1020, -0.6013, -1.1306]],
              
                      [[ 1.4982, -1.8341,  0.5934,  1.2519],
                       [-0.8765,  1.0085, -0.1124,  0.6715],
                       [-1.4403, -0.1020, -0.6013, -1.1306]],

In [26]:
%timeit Batch.stack(batches)

90.4 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.4982, -1.8341,  0.5934,  1.2519],
                      [-0.8765,  1.0085, -0.1124,  0.6715],
                      [-1.4403, -0.1020, -0.6013, -1.1306],
                      [ 1.4982, -1.8341,  0.5934,  1.2519],
                      [-0.8765,  1.0085, -0.1124,  0.6715],
                      [-1.4403, -0.1020, -0.6013, -1.1306],
                      [ 1.4982, -1.8341,  0.5934,  1.2519],
                      [-0.8765,  1.0085, -0.1124,  0.6715],
                      [-1.4403, -0.1020, -0.6013, -1.1306],
                      [ 1.4982, -1.8341,  0.5934,  1.2519],
                      [-0.8765,  1.0085, -0.1124,  0.6715],
                      [-1.4403, -0.1020, -0.6013, -1.1306],
                      [ 1.4982, -1.8341,  0.5934,  1.2519],
                      [-0.8765,  1.0085, -0.1124,  0.6715],
                      [-1.4403, -0.1020, -0.6013, -1.1306],
                      [ 1.4982, -1.8341,  0.5934,  1.2519],
                   

In [28]:
%timeit Batch.cat(batches)

166 µs ± 512 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

377 µs ± 2.58 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
